In [275]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Fonctions d'aide

cette section contient des méthodes dont nous avons besoin pour effectuer le prétraitement et le nettoyage des données.

Si vous trouvez un problème lié à nltk, vous pouvez exécuter ce code et télécharger les fichiers nécessaires

```python
import nltk
nltk.download()
```

In [165]:
def remove_extra_whitespace(string):
    string = re.sub(r'\s+', ' ', string)
    return re.sub(r"\s{2,}", " ", string).strip()

def remove_punctiation(string):
    return re.sub(
        r'[\u060C\u061B\u061F\u0660\u0661\u0662\u0663\u0664\u0665\u0666\u0667\u0668\u0669\u066A\u066B\u066C\u066D\u0640]',
        '', string)
def remove_dubplicated_letters(string):
    return re.sub(r'(.)\1{2,}', r'\1',string)

def remove_mentions(string):
    regex = re.compile(r"@[a-zA-Z0-9-_]+")
    return re.sub(regex, ' ', string)

def remove_hashtags(string):
    regex = re.compile("(?:^|\s)[＃#]{1}(\w+)", re.UNICODE)
    return re.sub(regex, ' ', string)

def remove_emojis(string):
    regex = re.compile(u'['
                         u'\U0001F300-\U0001F64F'
                         u'\U0001F680-\U0001F6FF'
                         u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    return re.sub(regex, ' ', string)


def remove_stop_words(string, stop_words=[]):
    words = []
    tokens = word_tokenize(string)
    for w in tokens:
        if w not in stop_words:
            words.append(w)
    return " ".join(words)
def number_of_urls(string):
    regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
    return len(re.findall(regex,string))

def number_of_hashtags(string):
    regex = re.compile(r'[#]')
    return len(re.findall(regex, string))

def number_of_mentions(string):
    regex = re.compile(r"@[a-zA-Z0-9-]+")
    return len(re.findall(regex, string))

def number_of_emojis(string):
    pattern = re.compile(u'['
                         u'\U0001F300-\U0001F64F'
                         u'\U0001F680-\U0001F6FF'
                         u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    iterator = re.findall(pattern, string)
    emojis = []
    for emoji in iterator:
        for m in emoji:
            emojis.append(m)
    return len(emojis) 

def remove_urls(string):
    regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
    return re.sub(regex, ' ', string)

def filter_data(data,max_mentions,max_urls,max_emojis,max_hashtags,min_length):
    data = data[(data.number_urls<= max_urls)&(data.number_mentions <= max_mentions)]
    data = data[(data.number_emojis <= max_emojis)&(data.number_hashtags <= max_hashtags)]
    return data[(data.length >= min_length)][['message']]

## Lire les données et ajouter des métadonnées

In [294]:
data = pd.read_json("stage.json")
data = data.rename(columns={0: 'message'})
data["number_hashtags"] = data["message"].apply(number_of_hashtags)
data["number_emojis"] = data["message"].apply(number_of_emojis)
data["length"] = data["message"].apply(len)
data["number_urls"] = data["message"].apply(number_of_urls)
data["number_mentions"] = data["message"].apply(number_of_mentions)

In [295]:
data.head(2)

,message,number_hashtags,number_emojis,length,number_urls,number_mentions
0,"Bon go faire les cartons, 8semaines pour démén...",0,0,60,0,0
1,ARK Coop #36 : NOUVELLE MAISON &amp; HALLOWEEN...,6,0,139,2,0


## Filtrer les données

Ces filtres vous pouvez utiliser sur les données brutes avant l'annotation. vous pouvez par exemple supprimer tous les tweets contenant plus de 2 hashtags ou plus d'un lien.

In [296]:
max_mentions = 100
max_urls = 100
max_emojis = 100
max_hashtags = 100
min_length = 0
data = filter_data(data,max_mentions,max_urls,max_emojis,max_hashtags,min_length)

In [297]:
data.head(2)

,message
0,"Bon go faire les cartons, 8semaines pour démén..."
1,ARK Coop #36 : NOUVELLE MAISON &amp; HALLOWEEN...


## Nettoyer les données

Vous pouvez choisir les méthodes à appliquer dans le traitement juste commenter ce que vous n'avez pas besoin.

In [298]:
def preprocessing(string):
    # remove urls
    string  = remove_urls(string) 
    # remove dubplicated letters
    string = remove_dubplicated_letters(string)
    # remove mentions 
    string = remove_mentions(string)
    # remove hashtasgs
    string = remove_hashtags(string)
    # remove punctiations
    #string = remove_punctiation(string)
    string = " ".join(word_tokenize(string, language='french'))
    # remove extra white spaces
    string = remove_extra_whitespace(string)
    return string

In [299]:
data["message"] = data["message"].apply(lambda string : string.replace('\'','’'))
data["nettoyer"] = data["message"].apply(preprocessing)
reg = '[!"%\'()̣*+,-./:;<=>?\[\]^_`{|}~1234567890#’”“′‘\\\]'
data["nettoyer"] = data["nettoyer"].apply(lambda string : re.sub(reg,' ', string))
regex = re.compile('[^a-zA-Z0-9àâéêèìôùûç _-’\']')
data["nettoyer"] = data["nettoyer"].apply(lambda string : regex.sub(' ', string))

## Tokenization
Après cette section, vous avez un message clair que vous pouvez utiliser pour chacune des étapes suivantes. l'atribut nettoyer a le message propre

In [300]:
french_stopwords =  stopwords.words('French')
# remove french stop words
data["nettoyer"] = data["nettoyer"].apply(lambda string : remove_stop_words(string , french_stopwords))
data.head(10)

,message,nettoyer
0,"Bon go faire les cartons, 8semaines pour démén...",Bon go faire les cartons semaines déménager pl...
1,ARK Coop #36 : NOUVELLE MAISON &amp; HALLOWEEN...,ARK Coop NOUVELLE MAISON amp HALLOWEEN
2,Voici l’état des lieux du parc de logements en...,Voici état lieux parc logements France
3,En cette fin d’année j’ai géré une nouvelle vi...,En cette fin année géré nouvelle vie professio...
4,En plus je la commence dans une nouvelle maiso...,En plus commence nouvelle maison nouveau cheva...
5,Donc j’me lève à 7h en vacance pour aller aide...,Donc lève h vacance aller aider mère peindre n...
6,#demenagement à Latour bas-Elne près de #Perpi...,Latour bas Elne près
7,"ok, j’y vois un peu plus clair dans l’empaquet...",ok vois peu plus clair empaquetage donc carton...
8,Bon bah pas de salle aujourd’hui. Ça va porter...,Bon bah salle aujourd hui Ça va porter cartons...
9,#JeVeuxUnQledTV car il me fait une tv pour not...,car fait tv nouvelle maison vacances
